<a href="https://colab.research.google.com/github/Viktoriia-kama/ML_hw4-hw16/blob/main/HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# Визначення шляху до папки з даними на Google Диску
base_path = '/content/drive/My Drive/data'
folders = ['idle', 'running', 'walking', 'stairs']

# Створення порожнього списку для зберігання всіх DataFrame
all_data = []

# Ітерація по всіх папках і файлах
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    print(f"Перевірка шляху: {folder_path}")
    if not os.path.exists(folder_path):
        print(f"Папка не знайдена: {folder_path}")
        continue  # Пропустити папку, яка не існує
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            df['activity'] = folder  # Додайте стовпець для ідентифікації активності
            all_data.append(df)

# Об'єднання всіх DataFrame в один
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)

    # Перегляд результатів
    print(combined_df.head())

    # Збереження об'єднаного DataFrame у новий CSV-файл
    combined_df.to_csv('/content/drive/My Drive/combined_data.csv', index=False)
else:
    print("Не знайдено жодного CSV-файлу для об'єднання.")

Перевірка шляху: /content/drive/My Drive/data/idle
Перевірка шляху: /content/drive/My Drive/data/running
Перевірка шляху: /content/drive/My Drive/data/walking
Перевірка шляху: /content/drive/My Drive/data/stairs
   accelerometer_X  accelerometer_Y  accelerometer_Z activity
0        -0.172383         4.922479         8.398860     idle
1        -0.158017         5.008670         8.250419     idle
2        -0.090980         4.989517         8.054095     idle
3        -0.229843         4.922479         8.523358     idle
4         0.014365         5.008670         8.283938     idle


In [32]:
df = combined_df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173850 entries, 0 to 173849
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   accelerometer_X  173850 non-null  float64
 1   accelerometer_Y  173850 non-null  float64
 2   accelerometer_Z  173850 non-null  float64
 3   activity         173850 non-null  object 
dtypes: float64(3), object(1)
memory usage: 5.3+ MB


In [74]:
df.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
accelerometer_X,173850.0,NaN,NaN,NaN,2.163539,8.903203,-39.188293,-2.94008,0.248997,5.525818,39.188293
accelerometer_Y,173850.0,NaN,NaN,NaN,1.584005,13.19985,-39.188293,-9.150639,-0.023942,9.983821,39.188293
accelerometer_Z,173850.0,NaN,NaN,NaN,0.915189,7.118025,-39.188293,-2.868254,0.057461,4.333505,39.188293
activity,173850,4,running,103140,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
X = df.drop(["activity"], axis =1)
y = df['activity']


In [63]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [64]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [65]:
# encoders = ce.OrdinalEncoder(cols=['activity'])

# y_train = encoders.fit_transform(y_train)
# y_test = encoders.fit_transform(y_test)
# y_train = np.ravel(y_train)
y_train

45560     running
81992     running
102101    running
90757     running
78330     running
           ...   
119879    walking
103694    running
131932    walking
146867    walking
121958    walking
Name: activity, Length: 116479, dtype: object

In [66]:
rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

In [72]:
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9994945181363407
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      3240
     running       1.00      1.00      1.00     33927
      stairs       1.00      0.99      1.00      1621
     walking       1.00      1.00      1.00     18583

    accuracy                           1.00     57371
   macro avg       1.00      1.00      1.00     57371
weighted avg       1.00      1.00      1.00     57371



In [71]:
svc = SVC()

rfc.fit(X_train, y_train)

y_pred1 = rfc.predict(X_test)

In [73]:
print(accuracy_score(y_test, y_pred1))
print(classification_report(y_test, y_pred1))

0.9994945181363407
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      3240
     running       1.00      1.00      1.00     33927
      stairs       1.00      0.99      1.00      1621
     walking       1.00      1.00      1.00     18583

    accuracy                           1.00     57371
   macro avg       1.00      1.00      1.00     57371
weighted avg       1.00      1.00      1.00     57371



Висновок: вище представлено дві моделі - алгоритми SVM та випадковий ліс. Обидва classification report показують нам високі метрики: повнота та точність дорівнюють 1, що в результаті дає метрику гармонійне серднє f1 рівним 1.
Отже, обидва методи працюють чудово, адже дають високі результати